In [2]:
from scipy.spatial.distance import pdist, squareform
from ripser import ripser
from itertools import product

import scipy
import ringity
import numpy as np
import os
import time
import networkx as nx

In [3]:
G = nx.erdos_renyi_graph(100, 0.1)

In [10]:
%%timeit
G = nx.erdos_renyi_graph(200, 0.1)
ringity.net_flow(G)

884 ms ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
G = nx.erdos_renyi_graph(200, 0.1)
ringity.current_distance(G)

905 ms ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
L = nx.laplacian_matrix(G).toarray()

C = np.zeros(L.shape)
C[1:,1:] = np.linalg.inv(L[1:,1:])

A = nx.adjacency_matrix(G)

N = G.number_of_nodes()
E = G.number_of_edges()
B = scipy.sparse.lil_matrix((E, N))

for ei, (u,v) in enumerate(G.edges):
    B[ei, u] = -1
    B[ei, v] = 1

F = B@C



In [104]:
%%timeit
edge_dict  = {}
for ei, e in enumerate(G.edges):
    F_ei = F[ei,:]
    rank = scipy.stats.rankdata(F_ei)
    edge_dict[e] = sum([(2*rank[j]-1-N)*F[ei,j] for j in range(N)])

122 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [107]:
%%timeit
ranks = map(scipy.stats.rankdata, F)
vectors = iter(F)
edge_dict2  = {}
for e in G.edges:
    edge_dict2[e] = np.sum((2*next(ranks)-1-N)*next(vectors))

41.6 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [108]:
%%timeit
ranks = map(scipy.stats.rankdata, F)
vectors = iter(F)
edge_dict3  = {e : np.sum((2*rank-1-N)*vector) for
                  e,rank,vector in zip(G.edges, ranks, vectors)}

56.5 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
edge_dict.keys() == edge_dict2.keys(), edge_dict2.keys() == edge_dict3.keys()

(True, True)

In [103]:
all([abs(v1-v2) < 10**(-10) for (v1,v2) in zip(edge_dict.values() , edge_dict2.values())]), \
all([abs(v1-v2) < 10**(-10) for (v1,v2) in zip(edge_dict2.values(), edge_dict3.values())])

(True, True)